In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
import io
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import pdb

BATCH_SIZE = 32

In [6]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    print(n, d)
    i = 0
    for line in fin:
        if i == VOCAB_SIZE:
            break
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(list(map(float, tokens[1:])))
        i += 1
    return data

In [7]:
from collections import Counter
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1
VOCAB_SIZE = 50000

def build_vocab():
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    word_vectors = pkl.load(open("fasttext_word_vectors.p", "rb"))
    id2token = list(word_vectors.keys())
    token2id = dict(zip(word_vectors, range(2,2+len(word_vectors)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return word_vectors, token2id, id2token

In [8]:
word_vectors, token2id, id2token = build_vocab()

In [9]:
_weights = np.array(list(word_vectors.values()))
pad_vec = np.zeros((1, 300))
unk_vec = np.random.randn(1, 300) * 0.01
pad_unk_vecs = np.vstack((pad_vec, unk_vec))
_WEIGHTS = np.vstack((pad_unk_vecs, _weights))
_WEIGHTS.shape

(50002, 300)

### Creating toy dataset with 5000 training data, 1000 validation

In [10]:
# snli_train = pd.read_csv('/Users/vrajiv/Desktop/rnn-cnn-natural-language-inference/hw2_data/snli_train.tsv', sep='\t')
snli_train = pd.read_csv('snli_train.tsv', sep='\t')
TRAIN_VAL_SPLIT = 99000
DATA_SIZE = 100000

In [11]:
sent1_data = list(snli_train["sentence1"])[:DATA_SIZE]
sent2_data = list(snli_train["sentence2"])[:DATA_SIZE]
data_label = list(snli_train["label"])[:DATA_SIZE]
print(len(sent1_data))

100000


In [12]:
snli_val = pd.read_csv('snli_val.tsv', sep='\t')
sent1_val_data = list(snli_val["sentence1"])
sent2_val_data = list(snli_val["sentence2"])
val_label_data = list(snli_val["label"])

In [13]:
def convert_labels_to_integers(data_label):
    for i in range(len(data_label)):
        if data_label[i] == "contradiction":
            data_label[i] = 0
        elif data_label[i] == "entailment":
            data_label[i] = 1
        elif data_label[i] == "neutral":
            data_label[i] = 2
    return data_label

In [14]:
data_label = convert_labels_to_integers(data_label)
val_label_data = convert_labels_to_integers(val_label_data)

In [15]:
SEED = 1
random.Random(SEED).shuffle(sent1_data)
random.Random(SEED).shuffle(sent2_data)
random.Random(SEED).shuffle(data_label)

In [16]:
def verify_order(sent1_data, sent2_data, data_label):
    i = random.randint(1, len(sent1_data))
    print(sent1_data[i])
    print(sent2_data[i])
    print(data_label[i])

verify_order(sent1_data, sent2_data, data_label)

A guy taking a photo of another guy who is standing in front of a broken television .
A woman is playing a video game .
0


In [17]:
verify_order(sent1_val_data, sent2_val_data, val_label_data)

A busy street scene on a rainy day full of umbrella toting walkers .
A bunch of people holding umbrellas on a rainy day .
1


### Train/Val split

In [18]:
sent1_train = sent1_data[:TRAIN_VAL_SPLIT]
sent2_train = sent2_data[:TRAIN_VAL_SPLIT]
train_label = data_label[:TRAIN_VAL_SPLIT]

sent1_val = sent1_val_data
sent2_val = sent2_val_data
val_label = val_label_data

In [19]:
print(len(sent1_train), len(sent2_train), len(train_label))
print(len(sent1_val), len(sent2_val), len(val_label))

99000 99000 99000
1000 1000 1000


In [20]:
verify_order(sent1_train, sent2_train, train_label)
verify_order(sent1_val, sent2_val, val_label)

A man is walking down a pavement path .
the man is in the shower
0
A woman stands at a podium with a slide show behind her .
A woman is standing at a podium .
1


### Word tokenizing

In [21]:
def tokenize(sentence_list):
    return [word_tokenize(sentence_list[i]) for i in range(len(sentence_list))]

# train
sent1_train_tokenized = tokenize(sent1_train)
sent2_train_tokenized = tokenize(sent2_train)

# val
sent1_val_tokenized = tokenize(sent1_val)
sent2_val_tokenized = tokenize(sent2_val)

### "One-hot encoding"
#### Represent each sentence as a vector of indices in our vocabulary

In [22]:
# convert token to id in the dataset
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data 

In [23]:
# train
sent1_train_indices = token2index_dataset(sent1_train_tokenized)
sent2_train_indices = token2index_dataset(sent2_train_tokenized)

# val
sent1_val_indices = token2index_dataset(sent1_val_tokenized)
sent2_val_indices = token2index_dataset(sent2_val_tokenized)

### Data Loader

In [24]:
MAX_SENTENCE_LENGTH = 30

import numpy as np
import torch
from torch.utils.data import Dataset

class TwoSentencesDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    """
    
    def __init__(self, sent1_data_list, sent2_data_list, target_list):
        """
        @param sent1_data_list: list of sentence1's (index matches sentence2's and target_list below)
        @param sent2_data_list: list of sentence2's
        @param target_list: list of correct labels

        """
        self.sent1_data_list = sent1_data_list
        self.sent2_data_list = sent2_data_list
        self.target_list = target_list
        assert (len(self.sent1_data_list) == len(self.target_list) and len(self.sent2_data_list) == len(self.target_list))

    def __len__(self):
        return len(self.sent1_data_list)
        
    def __getitem__(self, key):
        ###
        ### Returns [[sentence, 1, tokens], [sentence, 2, tokens]]
        ###
        """
        Triggered when you call dataset[i]
        """
        sent1_tokens_idx = self.sent1_data_list[key][:MAX_SENTENCE_LENGTH]
        sent2_tokens_idx = self.sent2_data_list[key][:MAX_SENTENCE_LENGTH]
        combined_tokens_idx = [sent1_tokens_idx, sent2_tokens_idx]
        label = self.target_list[key]
        return [combined_tokens_idx, len(sent1_tokens_idx), len(sent2_tokens_idx), label]

def twosentences_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    sent1_data_list = []
    sent2_data_list = []
    sent1_length_list = []
    sent2_length_list = []
    label_list = []
    combined_data_list = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[3])
        sent1_length_list.append(datum[1])
        sent2_length_list.append(datum[2])
    # padding
    for datum in batch:
        padded_vec_1 = np.pad(np.array(datum[0][0]), pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        padded_vec_2 = np.pad(np.array(datum[0][1]), pad_width=((0,MAX_SENTENCE_LENGTH-datum[2])), 
                                mode="constant", constant_values=0)
        combined_data_list.append([padded_vec_1, padded_vec_2])
    return [torch.from_numpy(np.array(combined_data_list)), 
            torch.LongTensor(sent1_length_list), torch.LongTensor(sent2_length_list), torch.LongTensor(label_list)]

In [25]:
train_dataset = TwoSentencesDataset(sent1_train_indices, sent2_train_indices, train_label)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           collate_fn=twosentences_collate_func,
                                           #shuffle=True
                                          )

val_dataset = TwoSentencesDataset(sent1_val_indices, sent2_val_indices, val_label)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           collate_fn=twosentences_collate_func,
                                           #shuffle=True
                                          )

### CNN Model

In [26]:
class CNN(nn.Module):
    def __init__(self, emb_size, hidden_size, num_layers, num_classes):

        super(CNN, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(_WEIGHTS))
    
        self.conv1 = nn.Conv1d(emb_size, hidden_size, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1)
        self.maxpool = nn.MaxPool1d(30)
        self.linear1 = nn.Linear(2*hidden_size, 100)
        self.linear2 = nn.Linear(100, num_classes)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x, sent1_lengths, sent2_lengths):
        
        batch_size = x.size()[0]
        seq_len = x.size()[2]
        
        sent1s = torch.tensor(x[:, 0, :]).cuda()
        sent2s = torch.tensor(x[:, 1, :]).cuda()
        ordered_sents = torch.cat([sent1s, sent2s], dim=0).cuda()

        embed = self.embedding(ordered_sents)
        hidden = self.conv1(embed.transpose(1,2)).transpose(1,2)
        hidden = F.relu(hidden.contiguous().view(-1, hidden.size(-1))).view(2*batch_size, seq_len, hidden.size(-1))

        hidden = self.conv2(hidden.transpose(1,2)).transpose(1,2)
        hidden = F.relu(hidden.contiguous().view(-1, hidden.size(-1))).view(2*batch_size, seq_len, hidden.size(-1))
        hidden = self.maxpool(hidden.transpose(1, 2)).transpose(1, 2).squeeze(dim=1)
        
        hidden_sent1s = hidden[0:batch_size, :]
        hidden_sent2s = hidden[batch_size:, :]     
        
        linear1 = self.linear1(torch.cat([hidden_sent1s, hidden_sent2s], dim=1))
        logits = self.linear2(linear1)
        return logits

In [27]:
# Function for testing the model
def test_model(loader, model):
    """
    Helper function that tests the model's performance on a dataset
    """
    correct = 0
    total = 0
    model.eval()
    for (data, sent1_lengths, sent2_lengths, labels) in loader:
        data_batch, sent1_length_batch, sent2_length_batch, label_batch = data.cuda(), sent1_lengths.cuda(), sent2_lengths.cuda(), labels.cuda()
        outputs = F.softmax(model(data_batch, sent1_length_batch, sent2_length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        labels = labels.cuda()
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

def train_model(model, lr = 0.001, num_epochs = 7, criterion = nn.CrossEntropyLoss()):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) 

    for epoch in range(num_epochs):
        for i, (data, sent1_lengths, sent2_lengths, labels) in enumerate(train_loader):
            model.train()
            data_batch, sent1_length_batch, sent2_length_batch, label_batch = data.cuda(), sent1_lengths.cuda(), sent2_lengths.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(data_batch, sent1_length_batch, sent2_length_batch)
            loss = criterion(outputs, label_batch)
            
            loss.backward()
            optimizer.step()
            # validate every 100 iterations
            if i > 0 and i % 100 == 0:
                # validate
                val_acc = test_model(val_loader, model)
                print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                           epoch+1, num_epochs, i+1, len(train_loader), val_acc))
                print('Epoch: [{}/{}], Step: [{}/{}], Training Loss: {}'.format( 
                           epoch+1, num_epochs, i+1, len(train_loader), loss))

In [28]:
model = CNN(emb_size = 300, hidden_size=200, num_layers=1, num_classes=3).cuda()
train_model(model)

Epoch: [1/7], Step: [101/3094], Validation Acc: 43.7
Epoch: [1/7], Step: [101/3094], Training Loss: 1.046177625656128
Epoch: [1/7], Step: [201/3094], Validation Acc: 49.4
Epoch: [1/7], Step: [201/3094], Training Loss: 0.9624378085136414
Epoch: [1/7], Step: [301/3094], Validation Acc: 48.8
Epoch: [1/7], Step: [301/3094], Training Loss: 0.9790204763412476
Epoch: [1/7], Step: [401/3094], Validation Acc: 51.7
Epoch: [1/7], Step: [401/3094], Training Loss: 0.9888293743133545
Epoch: [1/7], Step: [501/3094], Validation Acc: 52.4
Epoch: [1/7], Step: [501/3094], Training Loss: 0.7824007272720337
Epoch: [1/7], Step: [601/3094], Validation Acc: 57.4
Epoch: [1/7], Step: [601/3094], Training Loss: 1.1802796125411987
Epoch: [1/7], Step: [701/3094], Validation Acc: 57.8
Epoch: [1/7], Step: [701/3094], Training Loss: 0.8083867430686951
Epoch: [1/7], Step: [801/3094], Validation Acc: 56.7
Epoch: [1/7], Step: [801/3094], Training Loss: 0.7674963474273682
Epoch: [1/7], Step: [901/3094], Validation Acc: 5

Epoch: [3/7], Step: [1001/3094], Validation Acc: 62.4
Epoch: [3/7], Step: [1001/3094], Training Loss: 0.744685173034668
Epoch: [3/7], Step: [1101/3094], Validation Acc: 62.8
Epoch: [3/7], Step: [1101/3094], Training Loss: 0.7696452140808105
Epoch: [3/7], Step: [1201/3094], Validation Acc: 63.1
Epoch: [3/7], Step: [1201/3094], Training Loss: 0.6902296543121338
Epoch: [3/7], Step: [1301/3094], Validation Acc: 61.9
Epoch: [3/7], Step: [1301/3094], Training Loss: 0.9985197186470032
Epoch: [3/7], Step: [1401/3094], Validation Acc: 61.9
Epoch: [3/7], Step: [1401/3094], Training Loss: 0.9367342591285706
Epoch: [3/7], Step: [1501/3094], Validation Acc: 62.4
Epoch: [3/7], Step: [1501/3094], Training Loss: 0.7623982429504395
Epoch: [3/7], Step: [1601/3094], Validation Acc: 61.6
Epoch: [3/7], Step: [1601/3094], Training Loss: 0.7721515893936157
Epoch: [3/7], Step: [1701/3094], Validation Acc: 62.2
Epoch: [3/7], Step: [1701/3094], Training Loss: 0.8199191689491272
Epoch: [3/7], Step: [1801/3094], 

Epoch: [5/7], Step: [1901/3094], Validation Acc: 64.4
Epoch: [5/7], Step: [1901/3094], Training Loss: 0.6819231510162354
Epoch: [5/7], Step: [2001/3094], Validation Acc: 64.8
Epoch: [5/7], Step: [2001/3094], Training Loss: 0.5292691588401794
Epoch: [5/7], Step: [2101/3094], Validation Acc: 65.3
Epoch: [5/7], Step: [2101/3094], Training Loss: 0.7463434934616089
Epoch: [5/7], Step: [2201/3094], Validation Acc: 63.4
Epoch: [5/7], Step: [2201/3094], Training Loss: 0.6259404420852661
Epoch: [5/7], Step: [2301/3094], Validation Acc: 64.8
Epoch: [5/7], Step: [2301/3094], Training Loss: 0.546959400177002
Epoch: [5/7], Step: [2401/3094], Validation Acc: 64.1
Epoch: [5/7], Step: [2401/3094], Training Loss: 0.6889359951019287
Epoch: [5/7], Step: [2501/3094], Validation Acc: 62.4
Epoch: [5/7], Step: [2501/3094], Training Loss: 0.8621200323104858
Epoch: [5/7], Step: [2601/3094], Validation Acc: 64.3
Epoch: [5/7], Step: [2601/3094], Training Loss: 0.8099386692047119
Epoch: [5/7], Step: [2701/3094], 

Epoch: [7/7], Step: [2801/3094], Validation Acc: 62.1
Epoch: [7/7], Step: [2801/3094], Training Loss: 0.9613803625106812
Epoch: [7/7], Step: [2901/3094], Validation Acc: 63.6
Epoch: [7/7], Step: [2901/3094], Training Loss: 0.7707784175872803
Epoch: [7/7], Step: [3001/3094], Validation Acc: 64.1
Epoch: [7/7], Step: [3001/3094], Training Loss: 0.6876821517944336


In [29]:
###
### Ideas
###

# Dropout layers --> prob 0.5
# weight decay. 

#rnn --> layer normalize

# CNN masking
# do not backpropagate
# after conv, cresate tensor masked not update.
# right after regular linear layer
# 
# set all elements until padding to 1